<a href="https://colab.research.google.com/github/ijaycyndy/Data-Cleaning-Using-SQL/blob/main/Pozitive_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import numpy as np


import pandas as pd

# 1  read payment_month as plain text
df = pd.read_csv(
        "/content/sample_data/collab payment data.csv",
        dtype={"payment_month": "string"}
)

# 2  parse contract_end_date (DD/MM/YYYY)
df["contract_end_date"] = pd.to_datetime(
        df["contract_end_date"],
        dayfirst=True,
        errors="raise"
)

# 3  custom parser for the mixed payment_month strings
def parse_payment(s: str):
    s = s.strip()
    if "-" in s:
        # pattern like 2023-01-05  (Y-D-M)
        return pd.to_datetime(s, format="%Y-%d-%m", errors="coerce")
    elif "/" in s:
        # pattern like 01/05/2023  (D/M/Y)
        return pd.to_datetime(s, dayfirst=True, errors="coerce")
    else:
        return pd.NaT          # unexpected layout → NaT

df["payment_month"] = df["payment_month"].apply(parse_payment)

# ----- sanity check: raise if any payment_month failed to parse -----
if df["payment_month"].isna().any():
    bad_rows = df[df["payment_month"].isna()].head()
    raise ValueError(
        "Some payment_month strings could not be parsed. "
        f"Sample bad rows:\n{bad_rows}"
    )
# -------------------------------------------------------------------


# 4  retain raw text for audit
df["payment_month_raw"] = df["payment_month"].dt.strftime("%Y-%m-%d")

# 5  remove zeros & flag clawbacks
df = df[df["amount_paid"] != 0]
df["is_clawback"] = df["amount_paid"] < 0

df["contract_status"] = np.where(
    df["contract_end_date"] < pd.Timestamp("2025-07-01"),
    "inactive",
    "active"
)

df

/tmp/ipython-input-122-823520061.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["is_clawback"] = df["amount_paid"] < 0
/tmp/ipython-input-122-823520061.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["contract_status"] = np.where(


,supplier,mpan_mprn,contract_end_date,amount_paid,payment_month,Payment_Type,payment_month_raw,is_clawback,contract_status
0,Pozitive Energy,9341669006,2023-07-10,0.75,2023-05-01,Recurring,2023-05-01,False,inactive
1,Pozitive Energy,4232717110,2023-08-08,0.31,2024-05-01,Recurring,2024-05-01,False,inactive
3,Pozitive Energy,1824156410,2024-07-31,0.79,2023-08-01,Recurring,2023-08-01,False,inactive
5,Pozitive Energy,9341669006,2023-07-10,0.20,2022-11-01,Recurring,2022-11-01,False,inactive
7,Pozitive Energy,2520115710,2026-01-31,0.50,2025-04-01,Recurring,2025-04-01,False,active
...,...,...,...,...,...,...,...,...,...
23882,TotalEnergies,1100010974209,2026-03-31,579.67,2025-02-28,Recurring,2025-02-28,False,active
23883,TotalEnergies,1507954199998,2024-09-30,967.09,2024-09-30,Recurring,2024-09-30,False,inactive
23884,TotalEnergies,1416632240000,2025-12-31,86.86,2025-01-31,Recurring,2025-01-31,False,active
23885,TotalEnergies,2700001843909,2026-07-31,735.51,2025-02-28,Recurring,2025-02-28,False,active


In [ ]:
check = df[df["mpan_mprn"] == 1848605]

check

,supplier,mpan_mprn,contract_end_date,amount_paid,payment_month,payment_month_raw,is_clawback
1005,Pozitive Energy,1848605,2024-02-05,25.58,2024-01-01,2024-01-01,False
2439,Pozitive Energy,1848605,2024-02-05,30.13,2024-02-01,2024-02-01,False
2450,Pozitive Energy,1848605,2024-02-05,24.07,2024-02-01,2024-02-01,False
4956,Pozitive Energy,1848605,2024-02-05,88.14,2023-04-01,2023-04-01,False
5530,Pozitive Energy,1848605,2024-02-05,18.01,2024-04-01,2024-04-01,False
6423,Pozitive Energy,1848605,2024-02-05,2.52,2023-05-01,2023-05-01,False
8163,Pozitive Energy,1848605,2024-02-05,23.06,2023-06-01,2023-06-01,False
10086,Pozitive Energy,1848605,2024-02-05,15.32,2023-07-01,2023-07-01,False
11692,Pozitive Energy,1848605,2024-02-05,8.42,2023-08-01,2023-08-01,False
14209,Pozitive Energy,1848605,2024-02-05,11.11,2023-09-01,2023-09-01,False


column	meaning
gross_amount	sum of positive payments in that month
clawback_amount	sum of negative payments (refunds)
net_amount	gross + clawback (true cash effect)
num_payments	payment count

In [ ]:
# 1. Create month key
df["month_period"] = df["payment_month"].dt.to_period("M")

# 2. Aggregate per MPAN-month-supplier
monthly = (
    df.groupby(["supplier", "mpan_mprn", "month_period"])
      .agg(
          gross_amount    = ("amount_paid",   lambda s: s[s > 0].sum()),
          net_amount      = ("amount_paid",   "sum"),
          clawback_amount = ("amount_paid",   lambda s: s[s < 0].sum()),
          num_payments    = ("amount_paid",   "size")
      )
      .reset_index()
)

# 3. Build calendar per mpan-supplier up to June 2025
last_period = pd.Period("2025-06", freq="M")

rows = [
    pd.DataFrame({
        "mpan_mprn": g["mpan_mprn"].iloc[0],
        "supplier": g["supplier"].iloc[0],
        "month_period": pd.period_range(g["month_period"].min(), last_period, freq="M")
    })
    for _, g in monthly.groupby(["mpan_mprn", "supplier"])
]

calendar = pd.concat(rows, ignore_index=True)

# 4. Merge supplier info and contract details into calendar
calendar = calendar.merge(
    df[["mpan_mprn", "supplier", "contract_end_date", "Payment_Type", "contract_status"]]
      .drop_duplicates(),
    on=["mpan_mprn", "supplier"],
    how="left"
)

# 5. Merge monthly aggregation and fill blanks
payment_aggregation = (
    calendar.merge(monthly, on=["mpan_mprn", "supplier", "month_period"], how="left")
            .fillna({
                "gross_amount":    0,
                "net_amount":      0,
                "clawback_amount": 0,
                "num_payments":    0
            })
)

payment_aggregation


/tmp/ipython-input-123-2101174272.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["month_period"] = df["payment_month"].dt.to_period("M")


,mpan_mprn,supplier,month_period,contract_end_date,Payment_Type,contract_status,gross_amount,net_amount,clawback_amount,num_payments
0,1848605,EDF,2024-05,2025-02-05,Upfront,inactive,208.61,208.61,0.0,1.0
1,1848605,EDF,2024-06,2025-02-05,Upfront,inactive,0.00,0.00,0.0,0.0
2,1848605,EDF,2024-07,2025-02-05,Upfront,inactive,0.00,0.00,0.0,0.0
3,1848605,EDF,2024-08,2025-02-05,Upfront,inactive,0.00,0.00,0.0,0.0
4,1848605,EDF,2024-09,2025-02-05,Upfront,inactive,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
30220,2700008316565,Pozitive Energy,2025-02,2024-04-30,Recurring,inactive,0.00,0.00,0.0,0.0
30221,2700008316565,Pozitive Energy,2025-03,2024-04-30,Recurring,inactive,0.00,0.00,0.0,0.0
30222,2700008316565,Pozitive Energy,2025-04,2024-04-30,Recurring,inactive,0.00,0.00,0.0,0.0
30223,2700008316565,Pozitive Energy,2025-05,2024-04-30,Recurring,inactive,0.00,0.00,0.0,0.0


In [ ]:
check2 = train[train["mpan_mprn"] == 5297805]

check2

,mpan_mprn,month_period,gross_amount,net_amount,clawback_amount,num_payments
45,5297805,2023-10,31.48,31.48,0.00,2.0
46,5297805,2023-11,29.96,29.96,0.00,1.0
47,5297805,2023-12,72.04,72.04,0.00,1.0
48,5297805,2024-01,138.69,138.69,0.00,1.0
49,5297805,2024-02,163.77,163.77,0.00,1.0
50,5297805,2024-03,179.59,179.59,0.00,1.0
51,5297805,2024-04,144.75,144.75,0.00,1.0
52,5297805,2024-05,138.35,138.35,0.00,1.0
53,5297805,2024-06,100.31,100.31,0.00,1.0
54,5297805,2024-07,67.42,67.42,0.00,1.0


In [ ]:
# Sort for rolling logic
training_data = training_data.sort_values(["mpan_mprn", "supplier", "month_period"])

# 1. Month + Season
training_data["month_number"] = training_data["month_period"].dt.month
season_map = {
    12: "Winter", 1: "Winter", 2: "Winter",
    3: "Spring", 4: "Spring", 5: "Spring",
    6: "Summer", 7: "Summer", 8: "Summer",
    9: "Autumn", 10: "Autumn", 11: "Autumn"
}
training_data["season"] = training_data["month_number"].map(season_map)

# 2. Rolling Features — now grouped by both mpan and supplier
training_data["roll_3m_net"] = (
    training_data.groupby(["mpan_mprn", "supplier"])["net_amount"]
                 .rolling(window=3, min_periods=1)
                 .sum()
                 .reset_index(level=[0, 1], drop=True)
)

training_data["roll_3m_avg"] = (
    training_data.groupby(["mpan_mprn", "supplier"])["net_amount"]
                 .rolling(window=3, min_periods=1)
                 .mean()
                 .reset_index(level=[0, 1], drop=True)
)

training_data["roll_3m_gross"] = (
    training_data.groupby(["mpan_mprn", "supplier"])["gross_amount"]
                 .rolling(window=3, min_periods=1)
                 .sum()
                 .reset_index(level=[0, 1], drop=True)
)

# 3. Months since previous payment — grouped by both mpan and supplier
period_code = training_data["month_period"].astype(int)
training_data["months_since_prev_payment"] = (
    period_code.groupby([training_data["mpan_mprn"], training_data["supplier"]]).diff()
)

# 4. Reorder
cols_to_keep = [
    "mpan_mprn", "supplier", "month_period",
    "net_amount", "gross_amount", "clawback_amount", "num_payments",
    "contract_end_date", "contract_status", "Payment_Type",  # ← fix: add comma here
    "month_number", "season",
    "roll_3m_net", "roll_3m_avg", "roll_3m_gross",
    "months_since_prev_payment"
]


# Force rolling values to 0 for June 2025 rows where net_amount is 0

mask = (training_data["month_period"] == pd.Timestamp("2025-06-01")) & (training_data["net_amount"] == 0)
training_data.loc[mask, ["roll_3m_net", "roll_3m_avg"]] = 0

training_data = training_data[cols_to_keep]

training_data = training_data[
    (training_data["contract_status"] == "active") |
    (training_data["month_period"] <= training_data["contract_end_date"])
]


training_data

,mpan_mprn,supplier,month_period,net_amount,gross_amount,clawback_amount,num_payments,contract_end_date,contract_status,Payment_Type,month_number,season,roll_3m_net,roll_3m_avg,roll_3m_gross,months_since_prev_payment
0,1848605,EDF,2024-05-01,208.61,208.61,0.00,1.0,2025-02-05,inactive,Upfront,5,Spring,208.61,208.610000,208.61,NaN
1,1848605,EDF,2024-06-01,0.00,0.00,0.00,0.0,2025-02-05,inactive,Upfront,6,Summer,208.61,104.305000,208.61,2.678400e+15
2,1848605,EDF,2024-07-01,0.00,0.00,0.00,0.0,2025-02-05,inactive,Upfront,7,Summer,208.61,69.536667,208.61,2.592000e+15
3,1848605,EDF,2024-08-01,0.00,0.00,0.00,0.0,2025-02-05,inactive,Upfront,8,Summer,0.00,0.000000,0.00,2.678400e+15
4,1848605,EDF,2024-09-01,0.00,0.00,0.00,0.0,2025-02-05,inactive,Upfront,9,Autumn,0.00,0.000000,0.00,2.678400e+15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30206,2700008316565,Pozitive Energy,2023-12-01,13.83,13.83,0.00,1.0,2024-04-30,inactive,Recurring,12,Winter,13.85,4.616667,29.03,2.592000e+15
30207,2700008316565,Pozitive Energy,2024-01-01,10.68,10.68,0.00,1.0,2024-04-30,inactive,Recurring,1,Winter,24.51,8.170000,24.51,2.678400e+15
30208,2700008316565,Pozitive Energy,2024-02-01,-24.47,0.01,-24.48,2.0,2024-04-30,inactive,Recurring,2,Winter,0.04,0.013333,24.52,2.678400e+15
30209,2700008316565,Pozitive Energy,2024-03-01,2.96,2.96,0.00,1.0,2024-04-30,inactive,Recurring,3,Spring,-10.83,-3.610000,13.65,2.505600e+15


In [ ]:
training_data = training_data.sort_values(["mpan_mprn", "supplier", "month_period"])


training_data["next_month_payment"] = (
    training_data
    .groupby(["mpan_mprn", "supplier"])["net_amount"]
    .shift(-1)
)


training_data["will_pay_next_month"] = (
    training_data["next_month_payment"] > 0
).astype(int)


training_data


,mpan_mprn,supplier,month_period,net_amount,gross_amount,clawback_amount,num_payments,contract_end_date,contract_status,Payment_Type,month_number,season,roll_3m_net,roll_3m_avg,roll_3m_gross,months_since_prev_payment,next_month_payment,will_pay_next_month
0,1848605,EDF,2024-05-01,208.61,208.61,0.00,1.0,2025-02-05,inactive,Upfront,5,Spring,208.61,208.610000,208.61,NaN,0.00,0
1,1848605,EDF,2024-06-01,0.00,0.00,0.00,0.0,2025-02-05,inactive,Upfront,6,Summer,208.61,104.305000,208.61,2.678400e+15,0.00,0
2,1848605,EDF,2024-07-01,0.00,0.00,0.00,0.0,2025-02-05,inactive,Upfront,7,Summer,208.61,69.536667,208.61,2.592000e+15,0.00,0
3,1848605,EDF,2024-08-01,0.00,0.00,0.00,0.0,2025-02-05,inactive,Upfront,8,Summer,0.00,0.000000,0.00,2.678400e+15,0.00,0
4,1848605,EDF,2024-09-01,0.00,0.00,0.00,0.0,2025-02-05,inactive,Upfront,9,Autumn,0.00,0.000000,0.00,2.678400e+15,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30206,2700008316565,Pozitive Energy,2023-12-01,13.83,13.83,0.00,1.0,2024-04-30,inactive,Recurring,12,Winter,13.85,4.616667,29.03,2.592000e+15,10.68,1
30207,2700008316565,Pozitive Energy,2024-01-01,10.68,10.68,0.00,1.0,2024-04-30,inactive,Recurring,1,Winter,24.51,8.170000,24.51,2.678400e+15,-24.47,0
30208,2700008316565,Pozitive Energy,2024-02-01,-24.47,0.01,-24.48,2.0,2024-04-30,inactive,Recurring,2,Winter,0.04,0.013333,24.52,2.678400e+15,2.96,1
30209,2700008316565,Pozitive Energy,2024-03-01,2.96,2.96,0.00,1.0,2024-04-30,inactive,Recurring,3,Spring,-10.83,-3.610000,13.65,2.505600e+15,4.39,1


In [ ]:
check3 = training_data[
    (training_data["mpan_mprn"] == 10884307) &
    (training_data["supplier"] == "Pozitive Energy")
]

check3


,mpan_mprn,supplier,month_period,net_amount,gross_amount,clawback_amount,num_payments,contract_end_date,contract_status,Payment_Type,month_number,season,roll_3m_net,roll_3m_avg,roll_3m_gross,months_since_prev_payment,next_month_payment,will_pay_next_month
306,10884307,Pozitive Energy,2024-02-01,129.09,129.09,0.00,3.0,2026-11-11,active,Recurring,2,Winter,129.09,129.090000,129.09,NaN,90.50,1
307,10884307,Pozitive Energy,2024-03-01,90.50,90.50,0.00,1.0,2026-11-11,active,Recurring,3,Spring,219.59,109.795000,219.59,2.505600e+15,57.16,1
308,10884307,Pozitive Energy,2024-04-01,57.16,57.16,0.00,1.0,2026-11-11,active,Recurring,4,Spring,276.75,92.250000,276.75,2.678400e+15,50.49,1
309,10884307,Pozitive Energy,2024-05-01,50.49,50.49,0.00,1.0,2026-11-11,active,Recurring,5,Spring,198.15,66.050000,198.15,2.592000e+15,8.10,1
310,10884307,Pozitive Energy,2024-06-01,8.10,8.10,0.00,1.0,2026-11-11,active,Recurring,6,Summer,115.75,38.583333,115.75,2.678400e+15,0.95,1
311,10884307,Pozitive Energy,2024-07-01,0.95,0.95,0.00,1.0,2026-11-11,active,Recurring,7,Summer,59.54,19.846667,59.54,2.592000e+15,1.43,1
312,10884307,Pozitive Energy,2024-08-01,1.43,1.43,0.00,1.0,2026-11-11,active,Recurring,8,Summer,10.48,3.493333,10.48,2.678400e+15,0.95,1
313,10884307,Pozitive Energy,2024-09-01,0.95,0.95,0.00,1.0,2026-11-11,active,Recurring,9,Autumn,3.33,1.110000,3.33,2.678400e+15,6.19,1
314,10884307,Pozitive Energy,2024-10-01,6.19,6.19,0.00,1.0,2026-11-11,active,Recurring,10,Autumn,8.57,2.856667,8.57,2.592000e+15,1.43,1
315,10884307,Pozitive Energy,2024-11-01,1.43,1.43,0.00,1.0,2026-11-11,active,Recurring,11,Autumn,8.57,2.856667,8.57,2.678400e+15,26.20,1





MODEL TRAINING(for selecting the best hyper parameters)

In [ ]:

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error

# --- Classifier ---
clf = RandomForestClassifier(random_state=42)

param_dist_clf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

random_search_clf = RandomizedSearchCV(
    clf,
    param_distributions=param_dist_clf,
    n_iter=10,
    cv=3,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)

random_search_clf.fit(Xc_train, yc_train)

# Best classifier model
best_clf = random_search_clf.best_estimator_
y_pred_class = best_clf.predict(Xc_test)
accuracy = accuracy_score(yc_test, y_pred_class)
print("Best Classifier Accuracy:", round(accuracy * 100, 2), "%")
print("Best Classifier Params:", random_search_clf.best_params_)


# --- Regressor ---
reg = RandomForestRegressor(random_state=42)

param_dist_reg = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

random_search_reg = RandomizedSearchCV(
    reg,
    param_distributions=param_dist_reg,
    n_iter=10,
    cv=3,
    scoring='neg_root_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

random_search_reg.fit(Xr_train, yr_train)

# Best regressor model
best_reg = random_search_reg.best_estimator_
y_pred_reg = best_reg.predict(Xr_test)
rmse = np.sqrt(mean_squared_error(yr_test, y_pred_reg))
print("Best Regressor RMSE:", round(rmse, 2))
print("Best Regressor Params:", random_search_reg.best_params_)



KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error

model_data = training_data.copy()

# --- Encode binary features ---
model_data["is_upfront"] = (model_data["Payment_Type"].str.lower() == "upfront").astype(int)
model_data["contract_status_bin"] = (model_data["contract_status"].str.lower() == "active").astype(int)

# --- One-hot encode season and supplier ---
season_dummies = pd.get_dummies(model_data["season"], prefix="season")
supplier_dummies = pd.get_dummies(model_data["supplier"], prefix="supplier")
model_data = pd.concat([model_data, season_dummies, supplier_dummies], axis=1)

# --- Define feature columns ---
features = [
    "month_number",
    "roll_3m_net",
    "roll_3m_avg",
    "roll_3m_gross",
    "months_since_prev_payment",
    "contract_status_bin",
    "is_upfront"
] + list(season_dummies.columns) + list(supplier_dummies.columns)

# --- Drop missing target values ---
model_data = model_data.dropna(subset=["will_pay_next_month", "next_month_payment"])

# --- Classification Data ---
X_class = model_data[features]
y_class = model_data["will_pay_next_month"]
Xc_train, Xc_test, yc_train, yc_test = train_test_split(X_class, y_class, test_size=0.2, random_state=42)

# --- Regression Data (only for rows where payment is expected) ---
X_reg = model_data[model_data["will_pay_next_month"] == 1][features]
y_reg = model_data[model_data["will_pay_next_month"] == 1]["next_month_payment"]
Xr_train, Xr_test, yr_train, yr_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# ======================
# TRAIN CLASSIFIER (using best hyperparameters)
# ======================
clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=4,
    random_state=42
)
clf.fit(Xc_train, yc_train)
y_pred_class = clf.predict(Xc_test)
accuracy = accuracy_score(yc_test, y_pred_class)
print("Classification Accuracy:", round(accuracy * 100, 2), "%")

# ======================
# TRAIN REGRESSOR (using best hyperparameters)
# ======================
reg = RandomForestRegressor(
    n_estimators=200,
    max_depth=3,
    min_samples_split=2,
    min_samples_leaf=2,
    random_state=42
)
reg.fit(Xr_train, yr_train)
y_pred_reg = reg.predict(Xr_test)
rmse = np.sqrt(mean_squared_error(yr_test, y_pred_reg))
print("Regression RMSE:", round(rmse, 2))


Classification Accuracy: 85.65 %
Regression RMSE: 128.17


Forecasting Script : preparing the date for forecasting

In [ ]:
import pandas as pd

# Start with a copy
base_data = training_data.copy()

# STEP 1: Clean and standardize month_period
if isinstance(base_data["month_period"].iloc[0], pd.Period):
    base_data["month_period"] = base_data["month_period"].dt.to_timestamp()
else:
    base_data["month_period"] = pd.to_datetime(base_data["month_period"], errors="coerce")

# Drop rows with invalid month_period
base_data = base_data.dropna(subset=["month_period"])

# STEP 2: Filter to only active contracts
active_contracts = base_data[base_data["contract_status"].str.lower() == "active"]

# STEP 3: Get existing (mpan, month) combos to avoid duplicates
existing_mpan_months = set(zip(
    base_data["mpan_mprn"],
    base_data["month_period"].dt.to_period("M")
))

# STEP 4: Create new rows up to contract_end_date
future_rows = []

for _, row in active_contracts.iterrows():
    mpan = row["mpan_mprn"]
    start = row["month_period"] + pd.DateOffset(months=1)
    end = pd.to_datetime(row["contract_end_date"]).replace(day=1)
    forecast_months = pd.date_range(start=start, end=end, freq="MS")

    for date in forecast_months:
        period = date.to_period("M")
        if (mpan, period) in existing_mpan_months:
            continue  # Skip existing month

        new_row = {
            "mpan_mprn": mpan,
            "supplier": row["supplier"],
            "month_period": date,
            "month_number": date.month,
            "season": (
                "winter" if date.month in [12, 1, 2] else
                "spring" if date.month in [3, 4, 5] else
                "summer" if date.month in [6, 7, 8] else
                "autumn"
            ),
            "roll_3m_net": 0.0,
            "roll_3m_avg": 0.0,
            "roll_3m_gross": 0.0,
            "months_since_prev_payment": 1.0,
            "contract_status": row["contract_status"],
            "contract_end_date": row["contract_end_date"],
            "Payment_Type": row["Payment_Type"]
        }
        future_rows.append(new_row)

# STEP 5: Combine base and extended data
forecast_extension_df = pd.DataFrame(future_rows)
forecast_data = pd.concat([base_data, forecast_extension_df], ignore_index=True)

# STEP 6: Format month_period as YYYY-MM string
forecast_data["month_period"] = pd.to_datetime(forecast_data["month_period"], errors="coerce")
forecast_data = forecast_data.dropna(subset=["month_period"])
forecast_data["month_period"] = forecast_data["month_period"].dt.to_period("M").astype(str)

# # STEP 7: Drop unused columns before prediction
# forecast_data = forecast_data.drop(columns=[
#     "net_amount",
#     "gross_amount",
#     "clawback_amount",
#     "num_payments",
#     "next_month_payment",
#     "will_pay_next_month",
#     "contract_end_date"
# ])

forecast_data.reset_index(drop=True, inplace=True)
forecast_data = forecast_data.drop_duplicates(subset=["mpan_mprn", "month_period"])


# Final result
forecast_data


,mpan_mprn,supplier,month_period,net_amount,gross_amount,clawback_amount,num_payments,contract_end_date,contract_status,Payment_Type,month_number,season,roll_3m_net,roll_3m_avg,roll_3m_gross,months_since_prev_payment,next_month_payment,will_pay_next_month
0,1848605,EDF,2024-05,208.61,208.61,0.0,1.0,2025-02-05,inactive,Upfront,5,Spring,208.61,208.610000,208.61,NaN,0.0,0.0
1,1848605,EDF,2024-06,0.00,0.00,0.0,0.0,2025-02-05,inactive,Upfront,6,Summer,208.61,104.305000,208.61,2.678400e+15,0.0,0.0
2,1848605,EDF,2024-07,0.00,0.00,0.0,0.0,2025-02-05,inactive,Upfront,7,Summer,208.61,69.536667,208.61,2.592000e+15,0.0,0.0
3,1848605,EDF,2024-08,0.00,0.00,0.0,0.0,2025-02-05,inactive,Upfront,8,Summer,0.00,0.000000,0.00,2.678400e+15,0.0,0.0
4,1848605,EDF,2024-09,0.00,0.00,0.0,0.0,2025-02-05,inactive,Upfront,9,Autumn,0.00,0.000000,0.00,2.678400e+15,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196181,2700001843909,TotalEnergies,2026-03,NaN,NaN,NaN,NaN,2026-07-31,active,Recurring,3,spring,0.00,0.000000,0.00,1.000000e+00,NaN,NaN
196182,2700001843909,TotalEnergies,2026-04,NaN,NaN,NaN,NaN,2026-07-31,active,Recurring,4,spring,0.00,0.000000,0.00,1.000000e+00,NaN,NaN
196183,2700001843909,TotalEnergies,2026-05,NaN,NaN,NaN,NaN,2026-07-31,active,Recurring,5,spring,0.00,0.000000,0.00,1.000000e+00,NaN,NaN
196184,2700001843909,TotalEnergies,2026-06,NaN,NaN,NaN,NaN,2026-07-31,active,Recurring,6,summer,0.00,0.000000,0.00,1.000000e+00,NaN,NaN


In [ ]:
# ----------------------------
# FORECASTING SCRIPT

future_data = forecast_data.copy()

# 1. Feature engineering (same steps used in training)
future_data["is_upfront"] = (future_data["Payment_Type"].str.lower() == "upfront").astype(int)
future_data["contract_status_bin"] = (future_data["contract_status"].str.lower() == "active").astype(int)

# 2. One-hot encode season and supplier
season_dummies_future = pd.get_dummies(future_data["season"], prefix="season")
supplier_dummies_future = pd.get_dummies(future_data["supplier"], prefix="supplier")

# 3. Align columns with training one-hot structure
for col in season_dummies.columns:
    if col not in season_dummies_future:
        season_dummies_future[col] = 0
season_dummies_future = season_dummies_future[season_dummies.columns]

for col in supplier_dummies.columns:
    if col not in supplier_dummies_future:
        supplier_dummies_future[col] = 0
supplier_dummies_future = supplier_dummies_future[supplier_dummies.columns]

# 4. Concatenate all features
future_data = pd.concat([future_data, season_dummies_future, supplier_dummies_future], axis=1)

# 5. Final feature matrix for prediction
X_future = future_data[features]

# 6. Predict classification (whether supplier will pay)
future_data["will_pay_next_month"] = clf.predict(X_future)

# 7. Predict payment amount only for rows with expected payment
X_reg_future = future_data[future_data["will_pay_next_month"] == 1][features]
future_data.loc[future_data["will_pay_next_month"] == 1, "predicted_payment"] = reg.predict(X_reg_future)

# 8. Fill zero for rows with no expected payment
future_data["predicted_payment"] = future_data["predicted_payment"].fillna(0)

# 9. Final forecast output
forecast_result = future_data[[
    "supplier",
    "mpan_mprn",
    "month_period",
    "will_pay_next_month",
    "predicted_payment"
]]


forecast_result
# # Optional: sort or export
# print(forecast_result.head())
# # forecast_result.to_csv("predicted_supplier_payments.csv", index=False)


,supplier,mpan_mprn,month_period,will_pay_next_month,predicted_payment
0,EDF,1848605,2024-05,0,0.0
1,EDF,1848605,2024-06,0,0.0
2,EDF,1848605,2024-07,0,0.0
3,EDF,1848605,2024-08,0,0.0
4,EDF,1848605,2024-09,0,0.0
...,...,...,...,...,...
196181,TotalEnergies,2700001843909,2026-03,0,0.0
196182,TotalEnergies,2700001843909,2026-04,0,0.0
196183,TotalEnergies,2700001843909,2026-05,0,0.0
196184,TotalEnergies,2700001843909,2026-06,0,0.0


In [ ]:
forecast_result.to_csv("forecast_result.csv", index=False)

from google.colab import files
files.download("forecast_result.csv")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Statistical Analysis

In [ ]:
import pandas as pd

# Copy original data
df = forecast_data.copy()

# Step 1: Ensure date columns are in datetime format
df["month_period"] = pd.to_datetime(df["month_period"], errors="coerce")
df["contract_end_date"] = pd.to_datetime(df["contract_end_date"], errors="coerce")

# Step 2: Filter July 2025 onwards
july_2025 = pd.to_datetime("2025-07-01")
df_future = df[df["month_period"] >= july_2025].copy()

# Step 3: Separate recurring and upfront
recurring = df_future[df_future["Payment_Type"].str.lower() == "recurring"]
upfront = df[df["Payment_Type"].str.lower() == "upfront"].copy()


# Step 4: Recurring meters — Get last non-zero roll_3m_avg before July 2025
df_past = df[df["month_period"] < july_2025].copy()
last_roll_avg = (
    df_past[df_past["Payment_Type"].str.lower() == "recurring"]
    .sort_values(["mpan_mprn", "month_period"])
    .groupby("mpan_mprn")["roll_3m_avg"]
    .apply(lambda x: x[x > 0].iloc[-1] if (x > 0).any() else 0)
    .reset_index()
    .rename(columns={"roll_3m_avg": "forecast_amount"})
)
recurring_forecast = recurring.merge(last_roll_avg, on="mpan_mprn", how="left")

# Step 5: Upfront meters — Total paid per meter before July 2025
upfront_all = df[df["Payment_Type"].str.lower() == "upfront"]
upfront_total = (
    upfront_all[upfront_all["month_period"] < july_2025]
    .groupby("mpan_mprn")["net_amount"]
    .sum()
    .reset_index()
    .rename(columns={"net_amount": "total_paid"})
)
upfront_total["forecast_amount"] = upfront_total["total_paid"] * 0.2

# Merge year-month keys for matching contract end month
upfront["contract_ym"] = upfront["contract_end_date"].dt.to_period("M")
upfront["month_ym"] = upfront["month_period"].dt.to_period("M")

# Final month = contract end month (match on period, not exact date)
upfront_contract_month = upfront[
    upfront["contract_ym"] == upfront["month_ym"]
].merge(upfront_total[["mpan_mprn", "forecast_amount"]], on="mpan_mprn", how="left")

# All other months = 0
upfront_other_months = upfront[
    upfront["contract_ym"] != upfront["month_ym"]
].copy()
upfront_other_months["forecast_amount"] = 0

# Step 6: Combine
forecast_df = pd.concat([
    recurring_forecast,
    upfront_contract_month,
    upfront_other_months
], ignore_index=True)

# Step 7: Add pre-July 2025 data for active contracts
active_pre_july = df[
    (df["month_period"] < july_2025) &
    (df["contract_end_date"] >= july_2025) &  # still active in July 2025
    (df["Payment_Type"].str.lower() == "recurring")  # optional: only recurring if you prefer
].copy()

# Add a blank forecast column (or actuals if needed)
active_pre_july["forecast_amount"] = None

# Step 8: Combine all
forecast_df = pd.concat([
    active_pre_july,
    forecast_df
], ignore_index=True)






forecast_df


/tmp/ipython-input-137-699565457.py:75: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  forecast_df = pd.concat([


,mpan_mprn,supplier,month_period,net_amount,gross_amount,clawback_amount,num_payments,contract_end_date,contract_status,Payment_Type,...,season,roll_3m_net,roll_3m_avg,roll_3m_gross,months_since_prev_payment,next_month_payment,will_pay_next_month,forecast_amount,contract_ym,month_ym
0,10884307,Pozitive Energy,2024-02-01,129.09,129.09,0.0,3.0,2026-11-11,active,Recurring,...,Winter,129.09,129.090000,129.09,NaN,90.50,1.0,NaN,NaT,NaT
1,10884307,Pozitive Energy,2024-03-01,90.50,90.50,0.0,1.0,2026-11-11,active,Recurring,...,Spring,219.59,109.795000,219.59,2.505600e+15,57.16,1.0,NaN,NaT,NaT
2,10884307,Pozitive Energy,2024-04-01,57.16,57.16,0.0,1.0,2026-11-11,active,Recurring,...,Spring,276.75,92.250000,276.75,2.678400e+15,50.49,1.0,NaN,NaT,NaT
3,10884307,Pozitive Energy,2024-05-01,50.49,50.49,0.0,1.0,2026-11-11,active,Recurring,...,Spring,198.15,66.050000,198.15,2.592000e+15,8.10,1.0,NaN,NaT,NaT
4,10884307,Pozitive Energy,2024-06-01,8.10,8.10,0.0,1.0,2026-11-11,active,Recurring,...,Summer,115.75,38.583333,115.75,2.678400e+15,0.95,1.0,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24615,2380002924108,British Gas,2026-04-01,NaN,NaN,NaN,NaN,2026-05-05,active,Upfront,...,spring,0.00,0.000000,0.00,1.000000e+00,NaN,NaN,0.0,2026-05,2026-04
24616,2600000908418,EDF,2025-07-01,NaN,NaN,NaN,NaN,2025-11-30,active,Upfront,...,summer,0.00,0.000000,0.00,1.000000e+00,NaN,NaN,0.0,2025-11,2025-07
24617,2600000908418,EDF,2025-08-01,NaN,NaN,NaN,NaN,2025-11-30,active,Upfront,...,summer,0.00,0.000000,0.00,1.000000e+00,NaN,NaN,0.0,2025-11,2025-08
24618,2600000908418,EDF,2025-09-01,NaN,NaN,NaN,NaN,2025-11-30,active,Upfront,...,autumn,0.00,0.000000,0.00,1.000000e+00,NaN,NaN,0.0,2025-11,2025-09


In [ ]:
forecast_df.to_csv("forecast_df.csv", index=False)

from google.colab import files
files.download("forecast_df.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>